In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import scienceplots

import time
import math
import os
import sys
import random
from functools import partial
from decimal import Decimal
import numpy as np
import scipy.io as sio
import pysindy as ps
from tqdm import trange

sys.path.insert(0, '../')
from utils import *
from solvel0 import solvel0, MIOSR
from best_subset import backward_refinement, brute_force_all_subsets
from UBIC import *
from bayesian_model_evidence import log_evidence

from skimage.restoration import estimate_sigma
import bm3d
from kneed import KneeLocator

from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn import covariance
from sklearn.linear_model import lars_path
from abess import LinearRegression as AbessLinearRegression
from knockpy import KnockoffFilter, knockoff_stats, knockoffs
from knockpy.utilities import estimate_covariance
from scipy import stats
from statsmodels.stats.multitest import multipletests
from c2st.check import c2st # https://github.com/psteinb/c2st

from mbic import mbic, mbic2, ebic

from rdata import read_rds
from selective_inference import forward_stop_rule, sfs_si, stepwise_selective_inference, subset_fdr
import fpsample
from dppy.finite_dpps import FiniteDPP

from si4pipeline import (
                        construct_pipelines, 
                        extract_features, 
                        initialize_dataset, 
                        intersection, 
                        lasso, 
                        marginal_screening, 
                        stepwise_feature_selection, 
                        union, 
                        PipelineManager
                        )

alibi is not installed in the environment.


In [2]:
X_pre = np.load("../Cache/X_pre_burgers_noise50.npy")
y_pre = np.load("../Cache/y_pre_burgers_noise50.npy")
u_pre = y_pre.copy()
feature_names = np.load("../Cache/feature_names_burgers.npy")
fsInf = read_rds("../R/R_data/fsInf_screening_burgers_noise50.rds")

### Python

In [3]:
n_terms = 16
max_complexity = 12
alphas = [0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.01]

# _, lars_p, _ = lars_path(StandardScaler().fit_transform(X_pre), y_pre.flatten(), method='lasso', alpha_min=1e-6)
# lars_p = np.array(list(map(int, lars_p)))[:n_terms]

abess_nonzero = np.nonzero(AbessLinearRegression(path_type='gs', fit_intercept=False, alpha=1e-6, max_iter=100).fit(X_pre, y_pre.flatten()).coef_)[0]
_, lars_p, _ = lars_path(StandardScaler().fit_transform(X_pre[:, abess_nonzero]), y_pre.flatten(), method='lasso', alpha_min=0)
lars_p = abess_nonzero[np.array(list(map(int, lars_p)))]

X_test = X_pre[:, lars_p]
sigma = np.std(y_pre-X_test@np.linalg.lstsq(X_test, y_pre)[0], ddof=1)
manager = stepwise_selective_inference(support_size=len(lars_p))
_, p_list = manager.inference(X_test, y_pre, sigma)
print(lars_p, p_list, subset_fdr(p_list))

for alpha in alphas:
    adjusted_pvalues = p_list
    stop_step, false_discovery_rates = forward_stop_rule(adjusted_pvalues, alpha)
    adjusted_pvalues = adjusted_pvalues[:stop_step+1]
    rejections = np.sort(lars_p[:stop_step+1])
    if len(rejections) <= max_complexity: 
        break
max_fdr = alpha
max_fdr, feature_names[rejections], len(rejections)

[ 7 13  8 10 45 31  0  9 12 19  6 37 16 48 11 26] [0.005187067488279462, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09038630597012032, 0.4344178445540502, 0.0, 4.773959005888173e-15, 5.515118037813238e-06, 3.566748945504905e-05, 1.0063566424634018e-07, 0.0, 0.0006460122915141753, 0.0] 0.04190769176867432


(0.01,
 array(['x0_1', 'x0_11', 'x0_1111', 'x0x0_1', 'x0x0_1111', 'x0^3x0_111111'],
       dtype='<U13'),
 6)

### R

In [4]:
# max_complexity = 12
# alphas = [0.3, 0.2, 0.1, 0.05, 0.01]
# for alpha in alphas:
#     adjusted_pvalues = fsInf.get("pv")
#     stop_step, false_discovery_rates = forward_stop_rule(adjusted_pvalues, alpha)
#     adjusted_pvalues = adjusted_pvalues[:stop_step+1]
#     rejections = np.sort((fsInf.get("vars")-1).astype(np.int32)[:stop_step+1])
#     if len(rejections) <= max_complexity:
#         break
# max_fdr = alpha
# feature_names[rejections]

In [5]:
X_pre_top = X_pre[:, rejections]
X_pre_top = X_pre_top/np.linalg.norm(X_pre_top, 2, axis=0)
y_pre = u_pre.copy()

In [6]:
_, best_subsets = brute_force_all_subsets(X_pre_top, y_pre, max_support_size=8)
ebics = []
mbics = []
for _ in best_subsets:
    loglik = log_like_value(X_pre_top[:, _]@np.linalg.lstsq(X_pre_top[:, _], y_pre, rcond=None)[0], 
                            y_pre)
    ebics.append(ebic(loglik, len(_), len(y_pre), X_pre_top.shape[-1], const=0))
    mbics.append(mbic(loglik, len(_), len(y_pre), X_pre_top.shape[-1], const=2))
ebics = np.array(ebics)
mbics = np.array(mbics)

100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 39.01it/s]


In [7]:
# Assume that mbics is a decreasing sequence
complexities = np.array([len(_) for _ in best_subsets])
if np.alltrue(np.array(mbics) >= np.array([max(mbics)+_*(min(mbics)-max(mbics))/(np.argmin(mbics)-np.argmax(mbics)) for _ in range(len(best_subsets))])):
    knee = complexities.max()
else:
    decreasing_indices = np.array(mbics) <= np.array([max(mbics)+_*(min(mbics)-max(mbics))/(np.argmin(mbics)-np.argmax(mbics)) for _ in range(len(best_subsets))])
    knee = knee_finder(mbics[decreasing_indices])
    knee = (complexities[decreasing_indices])[knee]
knee

2

In [8]:
np.random.seed(0); random.seed(0)
n_samples = int(250*knee)
false_discovery_control_method = 'bh'
print("max fdr:", max_fdr)
for bs in best_subsets:
    fdrs = []
    for _ in range(len(y_pre)//n_samples):
        X_test = X_pre_top[:, bs]
        y_test = y_pre.ravel()
        
        np.random.seed(random.randint(0, 100))
        # sample_indices = sorted(set([np.random.randint(len(y_pre)) for _ in range(n_samples)]))
        sample_indices = fpsample.bucket_fps_kdline_sampling(X_test, n_samples=n_samples, h=3) # Farthest Point Sampling (FPS) is better!!!
        X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        # FPS + k-DPP
        DPP = FiniteDPP('likelihood', **{'L': X_test.dot(X_test.T)})
        DPP.flush_samples()
        for _ in range(n_samples//(len(bs))):
            DPP.sample_exact_k_dpp(size=len(bs))
        sample_indices = np.unique(np.ravel(DPP.list_of_samples))
        X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        
        manager = stepwise_selective_inference(support_size=X_test.shape[1])
        M, p_list = manager.inference(X_test, y_test, np.std(y_test))
        if false_discovery_control_method is not None:
            p_list = stats.false_discovery_control(p_list, method=false_discovery_control_method)
        # print(M, p_list, np.array(p_list) < 0.05)
        fdrs.append(subset_fdr(p_list))
        
    fdrs = np.array(fdrs)
    print(fdrs.mean(), stats.wilcoxon(fdrs-max_fdr, alternative='less').pvalue)

max fdr: 0.01
0.0 3.872108215522035e-06
6.044026390500346e-13 4.422457510605951e-05
0.056043083402590364 0.9999557754248939
0.15060642813102781 1.0
0.22212296361462328 0.9999558380706846
0.6002254801999019 0.9999557754248939


In [9]:
np.random.seed(0); random.seed(0)
n_samples = int(250*knee)
false_discovery_control_method = 'bh'
print("max fdr:", max_fdr)
for bs in best_subsets:
    fdrs = []
    for _ in range(len(y_pre)//n_samples):
        X_test = X_pre_top[:, bs]
        y_test = y_pre.ravel()
        
        np.random.seed(random.randint(0, 100))
        # sample_indices = sorted(set([np.random.randint(len(y_pre)) for _ in range(n_samples)]))
        sample_indices = fpsample.bucket_fps_kdline_sampling(X_test, n_samples=n_samples, h=3) # Farthest Point Sampling (FPS) is better!!!
        X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        # FPS + k-DPP
        # DPP = FiniteDPP('likelihood', **{'L': X_test.dot(X_test.T)})
        # DPP.flush_samples()
        # for _ in range(n_samples//(len(bs))):
        #     DPP.sample_exact_k_dpp(size=len(bs))
        # sample_indices = np.unique(np.ravel(DPP.list_of_samples))
        # X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        
        manager = stepwise_selective_inference(support_size=X_test.shape[1])
        M, p_list = manager.inference(X_test, y_test, np.std(y_test))
        if false_discovery_control_method is not None:
            p_list = stats.false_discovery_control(p_list, method=false_discovery_control_method)
        # print(M, p_list, np.array(p_list) < 0.05)
        fdrs.append(subset_fdr(p_list))
        
    fdrs = np.array(fdrs)
    print(fdrs.mean(), stats.wilcoxon(fdrs-max_fdr, alternative='less').pvalue)

max fdr: 0.01
0.0 3.872108215522035e-06
0.0 3.872108215522035e-06
0.02040027761285782 0.9999557754248939
0.06910577973320826 1.0
0.1253268016801939 0.9999558380706846
0.43978477842996944 0.9999557754248939
